In [1]:
import re
line = "Status of node 'rabbit@overcloud-controller-0' ..."

In [8]:
matchObj = re.match( r'Status of nodecds \'(.*)\' ...', line)

In [9]:
matchObj.group()

AttributeError: 'NoneType' object has no attribute 'group'

In [10]:
matchObj.group(1)

AttributeError: 'NoneType' object has no attribute 'group'

In [11]:
print matchObj

None


In [95]:
content = """
Status of node 'rabbit@overcloud-controller-2' ...
[{pid,4087},
 {running_applications,[{rabbit,"RabbitMQ","3.3.5"},
                        {os_mon,"CPO  CXC 138 46","2.2.14"},
                        {mnesia,"MNESIA  CXC 138 12","4.11"},
                        {xmerl,"XML parser","1.3.6"},
                        {sasl,"SASL  CXC 138 11","2.3.4"},
                        {stdlib,"ERTS  CXC 138 10","1.19.4"},
                        {kernel,"ERTS  CXC 138 10","2.16.4"}]},
 {os,{unix,linux}},
 {erlang_version,"Erlang R16B03-1 (erts-5.10.4) [source] [64-bit] [smp:32:32] [async-threads:30] [hipe] [kernel-poll:true]\n"},
 {memory,[{total,996824472},
          {connection_procs,39042152},
          {queue_procs,37838200},
          {plugins,0},
          {other_proc,15338592},
          {mnesia,7740960},
          {mgmt_db,0},
          {msg_index,1318936},
          {other_ets,2626760},
          {binary,859726712},
          {code,16696010},
          {atom,2378065},
          {other_system,14118085}]},
 {alarms,[]},
 {listeners,[{clustering,35672,"::"},{amqp,5672,"172.31.11.16"}]},
 {vm_memory_high_watermark,0.4},
 {vm_memory_limit,26943271731},
 {disk_free_limit,50000000},
 {disk_free,257304219648},
 {file_descriptors,[{total_limit,3996},
                    {total_used,835},
                    {sockets_limit,3594},
                    {sockets_used,833}]},
 {processes,[{limit,1048576},{used,13607}]},
 {run_queue,0},
 {uptime,3075474}]

Cluster status of node 'rabbit@overcloud-controller-2' ...
[{nodes,[{disc,['rabbit@overcloud-controller-0',
                'rabbit@overcloud-controller-1',
                'rabbit@overcloud-controller-2']}]},
 {running_nodes,['rabbit@overcloud-controller-0',
                 'rabbit@overcloud-controller-1',
                 'rabbit@overcloud-controller-2']},
 {cluster_name,<<"rabbit@overcloud-controller-0.localdomain">>},
 {partitions,[]}]

Consumers on test_vhost:

Permissions on /:
user    configure       write   read
guest   .*      .*      .*
redhat  redhat.*        .*      .*
redhat1 redhat.*        .*      .*

Permissions on test_vhost:

Policies on /:
"""

In [97]:
con = content.split('\n')
#print con
#for line in con:
#    print line

In [36]:
line0 = "user    configure   1996    write   read"
line1 = "guest   .*      .*      .*"
m0 = re.match(r'(.*)\s+(.*)\s+(.*)\s+(.*)', line1)

In [38]:
print m0.group(), m0.group(1), m0.group(2), m0.group(3)

guest   .*      .*      .* guest   .*      .*     


In [132]:
    fd_total_limit = {}
    permissions = {}

    
    def parse_content(content):
        mode = 'A'
        """
        permissions = { {"vhost-name": { ["username", "conf", "w", "r"],
                                         [ ]} },
                        {} }
        """
        # how to jump out of the mode if empty line
        for line in content:
            #print "line: ", line
            #print mode
            if mode == 'A':
                matchObj = re.match( r'Status of node \'(.*)\' ...', line)
                if matchObj:
                    node = matchObj.group(1)
                    mode = 'B'
                    continue
                matchObj = re.match( r'Permissions on (.*):', line)
                if matchObj:
                    vhost = matchObj.group(1)
                    permissions[vhost] = {}
                    mode = 'C'
                    continue
            elif mode == 'B':
                #{file_descriptors,[{total_limit,3996}  ,\[{total_limit,(\w+)},\]}
                if re.match(r'\s+{file_descriptors',line):
                    mode = 'B+'
                continue
            elif mode == 'B+':
                matchObj = re.search( r'{total_limit,(\w+)},', line)
                if matchObj:
                    fd_total_limit[node] = matchObj.group(1)
                    mode = 'A'
                continue
            elif mode == 'C':
                #print "XXXin mode C"
                if re.match(r'user\s+configure\s+write\s+read', line):
                    #print "XXXXmatch here"
                    #permissions[vhost] = {}
                    #print "XXXXchange mode C->D"
                    mode = 'D'
                else:
                    #print "XXXXchange mode C->A"
                    mode = 'A'
                continue
            elif mode == 'D':
                matchObj = re.match(r'(\S*)\s+(\S*)\s+(\S*)\s+(\S*)', line)
                if matchObj:
                    permissions[vhost][matchObj.group(1)] = [matchObj.group(2),
                            matchObj.group(3), matchObj.group(4)]
                    #print "XXXXpermit", permissions[vhost][matchObj.group(1)]
                else:# empty line
                    mode = 'A'
                continue


In [145]:
a = parse_content(con)

print "-----"
print fd_total_limit
print "-----"
print permissions

line:  
A
line:  Status of node 'rabbit@overcloud-controller-2' ...
A
line:  [{pid,4087},
B
line:   {running_applications,[{rabbit,"RabbitMQ","3.3.5"},
B
line:                          {os_mon,"CPO  CXC 138 46","2.2.14"},
B
line:                          {mnesia,"MNESIA  CXC 138 12","4.11"},
B
line:                          {xmerl,"XML parser","1.3.6"},
B
line:                          {sasl,"SASL  CXC 138 11","2.3.4"},
B
line:                          {stdlib,"ERTS  CXC 138 10","1.19.4"},
B
line:                          {kernel,"ERTS  CXC 138 10","2.16.4"}]},
B
line:   {os,{unix,linux}},
B
line:   {erlang_version,"Erlang R16B03-1 (erts-5.10.4) [source] [64-bit] [smp:32:32] [async-threads:30] [hipe] [kernel-poll:true]
B
line:  "},
B
line:   {memory,[{total,996824472},
B
line:            {connection_procs,39042152},
B
line:            {queue_procs,37838200},
B
line:            {plugins,0},
B
line:            {other_proc,15338592},
B
line:            {mnesia,7740960},
B
line:           

In [121]:
line2 = "   {file_descriptors,[{total_limit,3996},"
matchObj = re.match(r'\s+{file_descriptors,\[{total_limit,(\w+)}',line2)
#matchObj = re.match(r'\s+{file_descriptors,\[{total_limit,',line2)

In [122]:
print matchObj.group()

   {file_descriptors,[{total_limit,3996}


In [147]:
fd_content2 = """
Status of node 'rabbit@overcloud-controller-2' ...
[{pid,4087},
 {alarms,[]},
 {listeners,[{clustering,25672,"::"},{amqp,5672,"::"}]},
 {vm_memory_high_watermark,0.4},
 {vm_memory_limit,1590132736},
 {disk_free_limit,50000000},
 {disk_free,40712921088},
 {file_descriptors,
     [{total_limit,924},{total_used,3},{sockets_limit,829},{sockets_used,1}]},
 {processes,[{limit,1048576},{used,180}]},
 {run_queue,0},
 {uptime,5590}]
"""
con2 = fd_content2.split('\n')

In [148]:
for line in con2:
    if re.match(r'\s+{file_descriptors',line):
        matchObj = re.search(r'{total_limit,(\w+)},', line)
        print matchObj
    else:
        matchObj = re.search(r'{total_limit,(\w+)},', line)
        if matchObj:
            print matchObj
            print matchObj.group()
    

None
{total_limit,924},
